In [1]:
# Frame by Frame
import cv2
vidcap = cv2.VideoCapture("/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/Low04112021.mp4")
success, image = vidcap.read()
count = 0

while success:
    cv2.imwrite("/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/frames/frame%d.jpg" % count, image)
    success, image = vidcap.read()
    # print('Read a new frame: ', success)
    count += 1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
import cv2
import numpy as np
import time

# th architecture to use
arch = 'resnet50'
# load the pre-trained weights
model_file = '/raid/home/labuseraber/environment/testEnvironment/sceneDetection/models/%s_places365.pth.tar' % arch
model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

# load the class label
file_name = '/raid/home/labuseraber/environment/testEnvironment/sceneDetection/categories/categories_places365.txt'
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
frames = 0
lblDic = {}
start = time.time()

for x in range(1763):
    path = "/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/frames/frame" + str(x) + ".jpg"
    # print(path)
    frame = cv2.imread(path)
    # Read frame-by-frame
    frame = cv2.resize(frame,None,fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
    cv2_im = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2_im)
   
    input_img = V(centre_crop(img).unsqueeze(0))
    # forward pass
    logit = model.forward(input_img)
    h_x = F.softmax(logit, 1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    preds = ''
    horPos = 10
    verPos = 50
    label = []
    preds = "{} : {:.3f}".format(classes[idx[0]],probs[0])
    print(label)
    lblDic[frames] = label
    frames += 1
    print ("Frames : "+ str(frames))
    
    img = cv2.imread(path)
    print("Load ", img.shape)
    # name = str(preds) + " : " + str(out[0].max() * 100)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(img, preds, (15, 15), font, 0.5, (255, 255, 255), 1)
    r = cv2.imwrite("/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec" + str(x) + ".jpg", img)
    print("/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec" + str(x) + ".jpg")
    print("is stored")

[]
Frames : 1
Load  (1080, 1920, 3)
/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec0.jpg
is stored
[]
Frames : 2
Load  (1080, 1920, 3)
/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec1.jpg
is stored
[]
Frames : 3
Load  (1080, 1920, 3)
/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec2.jpg
is stored
[]
Frames : 4
Load  (1080, 1920, 3)
/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec3.jpg
is stored
[]
Frames : 5
Load  (1080, 1920, 3)
/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec4.jpg
is stored
[]
Frames : 6
Load  (1080, 1920, 3)
/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec5.jpg
is stored
[]
Frames : 7
Load  (1080, 1920, 3)
/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec6.jpg
is stored
[]
Fra

In [4]:
# Frames Building
import cv2

img_array = []

def toCheck(frame):
    x = 0
    if frame is None:
        return None

def openFrame(x):
    path = "/raid/home/labuseraber/environment/testEnvironment/sceneDetection/input/framesRec/frameRec" + str(x) + ".jpg"
    frame = cv2.imread(path)
    return frame
    
for x in range(561):
    frame = openFrame(x)
    if frame is None:
        x += 1
        print (x)
        toCheck(frame)
    else:   
        height, width, layers = frame.shape
        size = (width, height)
        img_array.append(frame)

        output = cv2.VideoWriter('/raid/home/labuseraber/environment/testEnvironment/sceneDetection/output/29122021RecFramesLOW.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
    output.write(img_array[i])
output.release()